In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

In [2]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "x_accel",
    "y_accel",
    "z_accel",
]

# Output classes to learn how to classify
LABELS = [
    "Walking", 
    "Jogging", 
    "Uptairs", 
    "Downstairs", 
    "Sitting", 
    "Standing"
] 

In [3]:
from numpy import genfromtxt
import numpy as np
import pandas as pd
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y = pd.read_csv('../Data/WISDM_y.csv')

print(x.shape, y.shape)

(1098209, 3) (1098208, 1)


In [4]:
def create_series(x,y,timestep,overlap):

    slide_step = int(timestep*(1-overlap))
    data_num = int((len(x)/slide_step)-1)
  
    dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
    labels = list()

    for i in range(data_num):
        labels.append(y[slide_step*(i+1)-1])
        for j in range(timestep):
            dataset[i,j,:] = x[slide_step*i+j,:]

    return dataset,np.array(labels)


In [5]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y).reshape(-1,1)


ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()


/Users/sandeep/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
dataset,labels = create_series(x,y,16,0.5)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [7]:
# Input Data 

training_data_count = len(X_train)  # 96092 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 41183 testing series
n_steps = len(X_train[0])  # 16 timesteps per series
n_input = len(X_train[0][0])  # 3 input parameters per timestep


# LSTM Neural Network's internal structure
# n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(41183, 16, 3) (41183, 6) 2.769982662671871 6.952643412201609
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()

# Flatten the input data
model.add(layers.Flatten(input_shape=(n_steps, n_input)))

# Dense layer equivalent to first GRU
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))

# Dense layer equivalent to second GRU
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))

# Dense layer equivalent to third GRU
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))

# Dense layer equivalent to fourth GRU
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.2))

# Output layer remains the same
model.add(layers.Dense(6, activation='softmax'))

model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])

history = model.fit(X_train,y_train,batch_size=64,epochs=10,validation_split=0.1,callbacks=[callback])

model.save('/Users/sandeep/Desktop/BUCourses/Project/saved_models/MLP')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 48)                0         
                                                                 
 dense (Dense)               (None, 256)               12544     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

INFO:tensorflow:Assets written to: /Users/sandeep/Desktop/BUCourses/Project/saved_models/MLP/assets


In [9]:
model.evaluate(X_test, y_test)

1287/1287 [==============================] - 0s 277us/step - loss: 0.2863 - accuracy: 0.9000


[0.28632891178131104, 0.8999587297439575]

In [10]:
import tensorflow as tf
from helper import compute_metrics_base, compute_metrics, measure_cpu_utilization_and_run
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [11]:
model_dir = '/Users/sandeep/Desktop/BUCourses/Project/saved_models/MLP'
model = tf.keras.models.load_model('/Users/sandeep/Desktop/BUCourses/Project/saved_models/MLP')
average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics_base, model, X_test, y_test, model_dir)

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


1287/1287 [==============================] - 0s 255us/step
Size of the model: 866.48 KB
Accuracy on the test set: 90.00%
Elapsed time: 0.49 seconds
Average CPU utilization: 18.55%


In [12]:
from helper import compute_metrics_base, compute_metrics, measure_cpu_utilization_and_run


In [13]:
#Implemented Dynamic Range Quantization

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_model_optimization.sparsity import keras as sparsity

#Load saved model
model = tf.keras.models.load_model('/Users/sandeep/Desktop/BUCourses/Project/saved_models/MLP')


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False

# Now, convert the model
mlp_dynamicQ_tflite_model = converter.convert()


# Save the quantized model to a file
with open('/Users/sandeep/Desktop/BUCourses/Project/saved_models/mlp_dynamicQ_model.tflite', 'wb') as f:
    f.write(mlp_dynamicQ_tflite_model)


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmprhtbhslf/assets


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmprhtbhslf/assets


In [14]:
average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics, mlp_dynamicQ_tflite_model, X_test, y_test, input_type='float32'
                                                )

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


Size of TFLite model: 59.95 KB


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Accuracy on the test set: 90.01%
Elapsed time: 0.37 seconds
Average CPU utilization: 15.85%
